In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [2]:
train = pd.read_csv('../input_data/new_train.csv')
test = pd.read_csv('../input_data/new_test.csv')

player = pd.read_csv('../input_data/player_prefecture.csv', header=None)
fifa_2014_mem = pd.read_csv('../input_data/fifa2014mem.csv')

In [3]:
train_x = train.drop(['y', 'id'], axis=1)
train_y = train['y']
test_x = test.drop(['id'], axis=1)

In [4]:
all_x = pd.concat([train_x, test_x])
all_x

,year,stage,match,gameday,time,home,away,stadium,tv,home_score,...,away_05,away_06,away_07,away_08,away_09,away_10,away_11,name,address,capa
0,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合,1,...,アレックス,青木 剛,増田 誓志,小笠原 満男,本山 雅志,大迫 勇也,ジュニーニョ,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
1,2012,Ｊ１,第３節第１日,03/24(土),13:03,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）,4,...,下平 匠,青木 拓矢,上田 康太,渡邉 大剛,曺 永哲,東 慶悟,ラファエル,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
2,2012,Ｊ１,第５節第１日,04/07(土),14:03,ベガルタ仙台,ジュビロ磐田,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ仙台／ＮＨＫ静岡,2,...,宮崎 智彦,山本 康裕,小林 裕紀,山田 大記,松浦 拓弥,菅沼 実,前田 遼一,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
3,2012,Ｊ１,第７節第１日,04/21(土),19:04,ベガルタ仙台,ＦＣ東京,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ ＢＳ１,4,...,太田 宏介,高橋 秀人,米本 拓司,石川 直宏,大竹 洋平,羽生 直剛,渡邉 千真,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
4,2012,Ｊ１,第１０節第１日,05/06(日),13:04,ベガルタ仙台,清水エスパルス,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ仙台,0,...,李 記帝,村松 大輔,アレックス,河井 陽介,ジミー フランサ,大前 元紀,高木 俊幸,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2014,Ｊ２,第４２節第１日,11/23(日・祝),14:04,ザスパクサツ群馬,愛媛ＦＣ,正田醤油スタジアム群馬,スカパー！／スカパー！プレミアムサービス,3,...,西岡 大輝,三原 向平,渡邊 一仁,吉村 圭司,原川 力,堀米 勇輝,河原 和寿,正田醤油スタジアム群馬,群馬県前橋市敷島町66,15135
309,2014,Ｊ２,第３５節第１日,10/04(土),13:03,Ｖ・ファーレン長崎,ファジアーノ岡山,長崎市総合運動公園かきどまり陸上競技場,スカパー！／スカパー！プレミアムサービス,1,...,田中 奏一,千明 聖典,上田 康太,三村 真,片山 瑛一,久保 裕一,ウーゴ,長崎市総合運動公園かきどまり陸上競技場,長崎県長崎市柿泊町2210,16000
310,2014,Ｊ２,第３８節第１日,10/26(日),14:03,Ｖ・ファーレン長崎,ロアッソ熊本,長崎市総合運動公園かきどまり陸上競技場,スカパー！／スカパー！プレミアムサービス,0,...,片山 奨典,中山 雄登,養父 雄仁,髙柳 一誠,嶋田 慎太郎,澤田 崇,齊藤 和樹,長崎市総合運動公園かきどまり陸上競技場,長崎県長崎市柿泊町2210,16000
311,2014,Ｊ２,第３６節第１日,10/11(土),13:03,コンサドーレ札幌,ジェフユナイテッド千葉,札幌厚別公園競技場,スカパー！／スカパー！プレミアムサービス,0,...,中村 太亮,佐藤 勇人,佐藤 健太郎,幸野 志有人,町田 也真人,谷澤 達也,森本 貴幸,札幌厚別公園競技場,北海道札幌市厚別区上野幌3条1-2-1,19620


- 特徴量作成

In [5]:
# rank関数
def rank(base_col, col_name, rank_arr):
    rank = 0
    all_x[col_name] = 0
    for i in rank_arr:
        all_x.loc[all_x[base_col]==i, col_name] = rank
        rank += 1
    return all_x

In [6]:
# 試合会場の都道府県の25歳以上の100人当たりのサッカー人口
address_col_num = all_x.columns.get_loc('address')
all_x['player_pref'] = 0
for i in range(len(player)):
    for x_i in range(len(all_x)):
        if player.iloc[i, 1] in all_x.iloc[x_i, address_col_num]:
            all_x.iloc[x_i, address_col_num + 2] = player.iloc[i, 3]
            
home_0_col_num = all_x.columns.get_loc('home_01')
away_11_col_num = all_x.columns.get_loc('away_11')

# 各試合のFIFA2014日本代表選手の出場選手人数
all_x['fifa_men_sum'] = 0
for i in range(len(fifa_2014_mem)):
    for mem in range(home_0_col_num, away_11_col_num + 1):
        for j in range(len(all_x)):
            if fifa_2014_mem.iloc[i, 0] == all_x.iloc[j, mem]:
                all_x.iloc[j, -1] += 1

In [7]:
all_x['week'] = all_x['gameday'].str[-2].replace(('月','火','水','木','金'), '平日')
all_x['iweek'] = 0
all_x.loc[all_x['week']=='土', 'iweek'] = 3
all_x.loc[all_x['week']=='日', 'iweek'] = 1
all_x.loc[all_x['week']=='祝', 'iweek'] = 2

all_x['match_day'] =all_x['match'].str.split('第', expand = True)[2].str.split('日',expand=True)[0].astype('int')
all_x['match'] =all_x['match'].str.split('第', expand = True)[1].str.split('節',expand=True)[0].astype('int')


all_x['gameday'] = all_x['gameday'].str.split('/', expand = True)[0].astype('int')

all_x['h_time'] = all_x['time'].str.split(':', expand = True)[0].astype('int')
all_x['m_time'] = all_x['time'].str.split(':', expand = True)[1].astype('int')

all_x['t_time'] = all_x.apply(lambda x: x['h_time'] * 60 + x['m_time'], axis=1)
all_x['pre_rank_t_time'] = np.floor(all_x['t_time'] /50) * 50
all_x['t_time'] = all_x['t_time'].max() - np.clip(all_x['t_time'], 782, 1144)

# rank関数を用いた特徴量
t_time_rank_arr = [1200.0, 750.0, 1050.0, 1150.0, 950.0, 1100.0, 1000.0, 800.0, 900.0, 850.0]
all_x = rank(base_col='pre_rank_t_time', col_name='t_time_rank', rank_arr=t_time_rank_arr)

time_rank_arr = [20, 12, 17, 15, 18, 13, 16, 14, 19]
all_x = rank(base_col='h_time', col_name='h_time_rank', rank_arr=time_rank_arr)

gameday_rank_arr = [12, 6, 10, 11, 9, 8, 7, 4, 3, 5]
all_x = rank(base_col='gameday', col_name='gameday_rank',  rank_arr=gameday_rank_arr)

pref_rank_arr =[2.38, 2.58, 2.68, 2.87, 1.68, 2.11, 2.13, 2.79,  2.0, 2.24, 2.91,
              2.82, 2.94,  3.4, 3.29, 2.52, 2.55, 2.67, 3.55, 3.91, 2.73, 3.26,
               4.5, 3.32, 4.35, 2.99, 2.09, 3.96, 3.68, 5.01]
all_x = rank(base_col='player_pref', col_name='pref_rank',  rank_arr=pref_rank_arr)

match_day_rank_arr = [4, 3, 1, 2]
all_x = rank(base_col='match_day', col_name='match_day_rank',  rank_arr=match_day_rank_arr)

# tvに関する特徴量
all_x['nhk_sogo'] = all_x.apply(lambda x: 1 if '総合' in x['tv'] else 0, axis=1)
all_x['tv_num'] = all_x.apply(lambda x: len(x['tv'].split('／')), axis=1)
all_x['record'] = all_x.apply(lambda x: 0 if '録' in x['tv'] else 1, axis=1)

all_x['temperature'] = np.floor(all_x['temperature'])
all_x['temperature'] = all_x.apply(lambda x:all_x['temperature'].max() - x['temperature'] if all_x['temperature'].median() < x['temperature'] else x['temperature'], axis=1)

In [8]:
all_x = all_x[[
    'capa',
    'stage',
    'weather',
    'stadium',
    'week', 
    'home',
    'away',
    'address',
    'fifa_men_sum',
    'iweek',
    't_time',
    'pre_rank_t_time',
    't_time_rank',
    'h_time_rank',
    'gameday_rank',
    'pref_rank',
    'match_day_rank',
    'nhk_sogo',
    'tv_num',
    'record',
    'temperature',
    'match',
]]

In [9]:
all_x.corr()

,capa,fifa_men_sum,iweek,t_time,pre_rank_t_time,t_time_rank,h_time_rank,gameday_rank,pref_rank,match_day_rank,nhk_sogo,tv_num,record,temperature,match
capa,1.000000,0.341397,0.206430,-0.024770,0.023078,0.181071,0.025202,0.025266,0.278232,0.041810,0.128391,0.077681,0.056070,0.004043,-0.084171
fifa_men_sum,0.341397,1.000000,0.229605,-0.067243,0.063092,0.199812,0.067979,0.026376,0.281440,0.017036,0.137817,0.087363,0.010445,-0.023938,-0.093439
iweek,0.206430,0.229605,1.000000,0.044244,-0.045921,0.262580,-0.067593,-0.124427,0.222753,0.022526,0.085662,0.071146,0.032078,-0.017731,-0.103193
t_time,-0.024770,-0.067243,0.044244,1.000000,-0.994961,-0.179830,-0.365843,0.176032,-0.084771,0.037635,0.088354,0.121656,-0.123366,0.225814,-0.153195
pre_rank_t_time,0.023078,0.063092,-0.045921,-0.994961,1.000000,0.184683,0.315225,-0.166111,0.086989,-0.034368,-0.094412,-0.131487,0.123656,-0.215306,0.143579
t_time_rank,0.181071,0.199812,0.262580,-0.179830,0.184683,1.000000,0.192648,-0.022211,0.205991,0.016521,0.107094,0.104780,0.055545,0.010025,-0.076465
h_time_rank,0.025202,0.067979,-0.067593,-0.365843,0.315225,0.192648,1.000000,0.003022,0.012591,0.035349,0.006964,0.022106,0.026060,-0.040778,-0.014883
gameday_rank,0.025266,0.026376,-0.124427,0.176032,-0.166111,-0.022211,0.003022,1.000000,0.007331,-0.005825,-0.009483,0.065889,-0.009194,-0.022276,-0.718969
pref_rank,0.278232,0.281440,0.222753,-0.084771,0.086989,0.205991,0.012591,0.007331,1.000000,0.012943,0.075238,-0.029506,0.135942,0.009968,-0.077717
match_day_rank,0.041810,0.017036,0.022526,0.037635,-0.034368,0.016521,0.035349,-0.005825,0.012943,1.000000,-0.011805,0.019770,0.027010,0.074858,-0.063953


In [10]:
all_x = pd.get_dummies(all_x, drop_first=True)
all_x

,capa,fifa_men_sum,iweek,t_time,pre_rank_t_time,t_time_rank,h_time_rank,gameday_rank,pref_rank,match_day_rank,...,address_茨城県鹿嶋市神向寺後山26-2,address_長崎県諫早市宇都町27-1,address_長崎県長崎市柿泊町2210,address_長野県松本市神林5300,address_静岡県磐田市新貝2500,address_静岡県袋井市愛野2300-1,address_静岡県静岡市清水区村松3880-1,address_香川県丸亀市金倉町830,address_鳥取県鳥取市蔵田423,address_鹿児島県鹿児島市与次郎2-2-2
0,19694,0,3,360,800.0,7,7,8,23,2,...,0,0,0,0,0,0,0,0,0,0
1,19694,0,3,421,750.0,1,5,8,23,2,...,0,0,0,0,0,0,0,0,0,0
2,19694,0,3,361,800.0,7,7,7,23,2,...,0,0,0,0,0,0,0,0,0,0
3,19694,2,3,60,1100.0,5,8,7,23,2,...,0,0,0,0,0,0,0,0,0,0
4,19694,0,1,420,750.0,1,5,9,23,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,15135,0,2,360,800.0,7,7,3,3,2,...,0,0,0,0,0,0,0,0,0,0
309,16000,0,3,421,750.0,1,5,2,4,2,...,0,0,1,0,0,0,0,0,0,0
310,16000,0,1,361,800.0,7,7,2,4,2,...,0,0,1,0,0,0,0,0,0,0
311,19620,0,3,421,750.0,1,5,2,16,2,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# データの再分割
train_x = all_x.iloc[:train_x.shape[0], :].reset_index(drop=True)
test_x = all_x.iloc[train_x.shape[0]:, :].reset_index(drop=True)

In [12]:
model = LinearRegression()

In [13]:
# ホールドアウト
train_val_x, valid_x, train_val_y, valid_y = train_test_split(train_x,train_y ,test_size=0.2,random_state=17)

model.fit(train_val_x, train_val_y)
valid_pred = model.predict(valid_x)
print(np.sqrt(mean_squared_error(valid_y, valid_pred)))

3028.702214860955


In [14]:
model.fit(train_x, train_y)

LinearRegression()

In [15]:
pred = model.predict(test_x)

In [16]:
# データをcsv形式に加工。
out = pd.DataFrame({'id':test['id'], 'y':pred})

# idでソート。
out_sorted = out.sort_values('id')

out_sorted.to_csv('../submit/submit_linear_regression.csv', sep=',', header=False, index=False)

In [17]:
pd.read_csv('../submit/submit_linear_regression.csv')

,15822,14361.703752992536
0,15823,16776.283662
1,15824,35386.832504
2,15825,13343.550481
3,15827,28478.650607
4,15828,14010.399010
...,...,...
307,16432,6550.040220
308,16433,10740.156135
309,16434,6383.075184
310,16435,8243.392337
